# Projet P7 : Implémentez un modèle de scoring

## Contexte

Vous êtes Data Scientist au sein d'une société financière, nommée "Prêt à dépenser", qui propose des crédits à la consommation pour des personnes ayant peu ou pas du tout d'historique de prêt.

L’entreprise souhaite mettre en œuvre un outil de “scoring crédit” pour calculer la probabilité qu’un client rembourse son crédit, puis classifie la demande en crédit accordé ou refusé. Elle souhaite donc développer un algorithme de classification en s’appuyant sur des sources de données variées (données comportementales, données provenant d'autres institutions financières, etc.)

<b> MISSION : </b>

Construire un modèle de scoring qui donnera une prédiction sur la probabilité de faillite d'un client de façon automatique.

Analyser les features qui contribuent le plus au modèle, d’une manière générale (feature importance globale) et au niveau d’un client (feature importance locale), afin, dans un soucis de transparence, de permettre à un chargé d’études de mieux comprendre le score attribué par le modèle.

Mettre en production le modèle de scoring de prédiction à l’aide d’une API et réaliser une interface de test de cette API.

<b> Approche MLOps : </b>

Afin de pouvoir faire évoluer régulièrement le modèle, mettre en œuvre une démarche de type MLOps d’automatisation et d’industrialisation de la gestion du cycle de vie du modèle (du tracking des expérimentations à l’analyse en production du data drift). 

Mettre en oeuvre au minimum les étapes orientées MLOps suivantes : 

- Dans le notebook d’entraînement des modèles, générer à l’aide de MLFlow un tracking d'expérimentations
- Lancer l’interface web 'UI MLFlow" d'affichage des résultats du tracking
- Réaliser avec MLFlow un stockage centralisé des modèles dans un “model registry”
- Tester le serving MLFlow
- Gérer le code avec le logiciel de version Git
- Partager le code sur Github pour assurer une intégration continue
- Utiliser Github Actions pour le déploiement continu et automatisé du code de l’API sur le cloud
- Concevoir des tests unitaires avec Pytest (ou Unittest) et les exécuter de manière automatisée lors du build réalisé par Github Actions

<b> Elaboration du modèle : </b>

Attention à deux points spécifiques au contexte métier : 

- Le déséquilibre entre le nombre de bons et de moins bons clients doit être pris en compte pour élaborer un modèle pertinent, avec une méthode au choix
- Le déséquilibre du coût métier entre un faux négatif (FN - mauvais client prédit bon client : donc crédit accordé et perte en capital) et un faux positif (FP - bon client prédit mauvais : donc refus crédit et manque à gagner en marge). Vous pourrez supposer, par exemple, que le coût d’un FN est dix fois supérieur au coût d’un FP. Vous créerez un score “métier” (minimisation du coût d’erreur de prédiction des FN et FP) pour comparer les modèles, afin de choisir le meilleur modèle et ses meilleurs hyperparamètres. Attention cette minimisation du coût métier doit passer par l’optimisation du seuil qui détermine, à partir d’une probabilité, la classe 0 ou 1 (un “predict” suppose un seuil à 0.5 qui n’est pas forcément l’optimum). En parallèle, maintenez pour comparaison et contrôle des mesures plus techniques, telles que l’AUC et l’accuracy.

## Etapes du projet :

Elaboration d'un modèle de prédiction sous forme d’une API qui permet de calculer la probabilité de défaut du client, ainsi que sa classe (accepté ou refusé), déployer l'API sur une plateforme Cloud.

- <b> Etape préliminaire : </b> Importation des données ( --> NoteBook1 : Preprocessing )
- <b> Etape 1 : </b> EDA sur la table principale application ( --> NoteBook1 : Preprocessing ) 
- <b> Etape 2 : </b> Ajout des tables bureau et bureau_balance ( --> NoteBook1 : Preprocessing)
- <b> Etape 3 : </b> Ajout des tables previous_application, POS_CASH_balance, installments_payments et credit_card_balance ( --> NoteBook1 : Preprocessing)
- <b> Etape 4 : </b> Feature selection, feature engineering ( --> NoteBook1 : Preprocessing)
- <b> Etape 5 : </b> Elaboration des modèles avec un tracking d'expérimentations ( --> NoteBook2 : MLFlow_Modeles)
- <b> Etape 6 : </b> Création de l'API et des tests unitaires ( --> NoteBook3 : Creation_API & scripts app.py et test_api.py)
- <b> Etape 7 : </b> Déploiement de l’API sur une plateforme Cloud (Azure)
- <b> Etape 8 : </b> Analyse du Data Drift (--> NoteBook4 : DataDRIFT)

## Imports

In [7]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn methods
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

from flask import Flask, request, jsonify
import joblib

# Outils MLOps
import mlflow
import mlflow.sklearn

## ETAPE 6 : Création de l'API

ETAPE 6 : Création d'un fichier .py séparé pour notre application Flask = ce fichier importera le modèle et définira les endpoints pour prédire les résultats.
    
    1. Importer les données clients à prédire 
    2. Transformer les données 
        - standardisation 
        - imputation
    3. Création de l'API avec Flask (fichier python app.py)
    4. Intéroger l'API en local 
    5. Intéroger l'API sur le web
    6. Mettre en place un test de l'API avec unittest (fichier python test_api.py)

### 1. Importer les données clients à prédire 

In [11]:
data_client = pd.read_csv('./data/test_small.csv')

In [13]:
# Sélectionner un échantillon aléatoire de 100 clients pour commiter le fichier (github)
subset_clients = data_client.sample(n=100, random_state=42)
subset_clients.to_csv('./data/subset_clients.csv', index = False)

In [15]:
subset_clients.set_index('SK_ID_CURR', inplace=True)

In [17]:
subset_clients.shape

(100, 276)

In [23]:
# Sauvegarder les noms des colonnes avant transformation
column_names = subset_clients.columns

### 2. Transformer les données 

#### -- Standardiser --

In [27]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(subset_clients)

#### -- Imputer les valeurs manquantes --

In [30]:
# Créer une instance de SimpleImputer avec la stratégie 'median'
imputer = SimpleImputer(strategy='median')

# Appliquer l'imputer sur X_scaled
X_scaled_imputed = imputer.fit_transform(X_scaled)

In [32]:
# Transformer X_scaled_imputed en DataFrame et récupérer les colonnes d'origine
X_scaled_imputed_df = pd.DataFrame(X_scaled_imputed, index=subset_clients.index, columns=column_names)

### 3. Création de l'API

Cette partie n'est pas exécutée ici mais dans un script app.py

In [ ]:
# Tester l'API en local
# Pour le fichier app.py il faudra modifier l'url et le chemin du read.csv

# Imports
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, make_scorer
import joblib
from flask import Flask, jsonify, request
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from utils_fonction_cout import cout_metier
from utils_threshold import find_best_threshold


# Importer les données clients à prédire 
data_client = pd.read_csv('./data/subset_clients.csv')
# mettre la colonne id client en index
data_client.set_index('SK_ID_CURR', inplace=True)
# Sauvegarder les noms des colonnes avant transformation
column_names = data_client.columns

# Transformer les données - centrer et réduire
scaler = StandardScaler()
X_scaled = scaler.fit_transform(data_client)

# Transformer les données - Imputer les valeurs manquantes avec la stratégie 'median'
imputer = SimpleImputer(strategy='median')
X_scaled_imputed = imputer.fit_transform(X_scaled)

# Transformer X_scaled_imputed en DataFrame et récupérer les colonnes d'origine
X_scaled_imputed_df = pd.DataFrame(X_scaled_imputed, index=data_client.index, columns=column_names)

# Charger le modèle 
logistic_model = joblib.load('logistic_model.pkl')

# Initialiser l'application Flask
app = Flask(__name__)

# Définir un endpoint pour la prédiction
@app.route('/predict', methods=['GET'])
def predict():
    # Récupérer le client_id depuis les paramètres de la requête
    client_id = request.args.get('client_id', type=int)
    
    if client_id is None:
        return jsonify({"error": "client_id manquant ou invalide"}), 400
    
    # Récupérer les données du client
    client = X_scaled_imputed_df.loc[client_id]
    
    # Extraire les caractéristiques et les convertir en tableau NumPy
    features = np.array(client).reshape(1, -1)
    
    # Faire une prédiction de défaut de paiement
    prediction = logistic_model.predict(features)
    
    # Vérifier le résultat de la prédiction et retourner le message approprié
    result = "credit accepte" if int(prediction[0]) == 0 else "credit refuse"
    
    # La probabilité de défaut
    probability_class_1 = logistic_model.predict_proba(features)[:, 1]
    
    # Convertir la probabilité en liste pour JSON
    probability_class_1_list = probability_class_1.tolist()
    
    # Retourner le résultat sous forme de JSON
    return jsonify({
        'client_id': client_id,
        'probabilite_defaut (seuil=0.5)': probability_class_1_list,
        'prediction': result
    })

# Définir un endpoint de test pour s'assurer que l'API fonctionne
@app.route('/', methods=['GET'])
def home():
    return "API de prédiction de régression logistique est opérationnelle!"

# Lancer l'application Flask
if __name__ == '__main__':
    app.run(debug=True)

### 4. Intéroger l'API en local

In [100]:
import requests

url = 'http://127.0.0.1:5000/predict?client_id=418609'

# Utiliser requests.get() pour envoyer une requête GET
response = requests.get(url)

# Afficher la réponse JSON
print(response.json())

{'client_id': 418609, 'prediction': 'credit accepte', 'probabilite_defaut (seuil=0.5)': [0.22180146978918214]}


### 5. Intéroger l'API sur le web

In [98]:
# Après déploiement de l'API
# Liste de clients pour test (acceptés) : 291599, 418609, 202661, 185171, 111761, 258225, 174954, 297336, 369893, 195695, 384221, 203868, 182895
# Liste de clients pour test (refusés) : 362707, 398791, 195695

import requests

url = 'https://webappscoringcredit-gcbhe8axc2exdfge.francecentral-01.azurewebsites.net/predict?client_id=398791'

# Utiliser requests.get() pour envoyer une requête GET
response = requests.get(url)

# Afficher la réponse JSON
print(response.json())

{'client_id': 398791, 'prediction': 'credit refuse', 'probabilite_defaut (seuil=0.5)': [0.9061315323273397]}


### 6. Mettre en place un test de l'API

Cette partie n'est pas exécutée ici mais dans un script test_api.py

In [ ]:
# Tester l'API en local
# Pour le fichier test_api.py il faudra modifier l'url et le chemin du read.csv

import unittest
import requests

# URL de ton API
BASE_API_URL = "http://127.0.0.1:5000/predict"

class TestPredictAPI(unittest.TestCase):
        
    def test_predict_random_client(self):
        """Test de prédiction pour un client du fichier"""
        id_client = 144092  # Test pour un id_client
        url = f"{BASE_API_URL}?client_id={id_client}"  # Construire l'URL avec le paramètre client_id
        
        # Envoyer une requête GET à l'API
        response = requests.get(url)

        # Vérifie que la requête a réussi
        self.assertEqual(response.status_code, 200)
        
        # Vérifie que la réponse est au format JSON et contient les clés appropriées
        response_data = response.json()
        self.assertIn('client_id', response_data)
        self.assertIn('probabilite_defaut (seuil=0.5)', response_data)
        self.assertIn('prediction', response_data)
        
        # Vérifie que la prédiction est soit "crédit accepté", soit "crédit refusé"
        prediction = response_data['prediction']
        self.assertIn(prediction, ["credit accepte", "credit refuse"])
        
        # Vérifie que 'probabilite_defaut (seuil=0.5)' est une liste de nombres
        self.assertIsInstance(response_data['probabilite_defaut (seuil=0.5)'], list)
        self.assertTrue(all(isinstance(x, float) for x in response_data['probabilite_defaut (seuil=0.5)']))

if __name__ == '__main__':
    unittest.main()